In [1]:
import qcodes as qc

In [2]:
import numpy as np
import time

In [3]:
from qcodes.instrument_drivers.NI.PXI_6251 import PXI_6251, MetaXYScannerDriver, trajectory, dist
from nidaqmx.constants import AcquisitionType, RegenerationMode, UnderflowBehavior, TriggerType, Level
from nidaqmx.types import CtrFreq

In [4]:
def scan_pattern(x_pos_list, y_pos_list):
    xy_scan_list=[]
    for j, y in enumerate(y_pos_list):
        for x in x_pos_list:
            xy_scan_list.append([x, y])
    return xy_scan_list

In [5]:
daq_outputs = PXI_6251('daq_outputs', 'PXI-6713')

In [8]:
daq_outputs._terminals

['/PXI-6713/PFI0',
 '/PXI-6713/PFI1',
 '/PXI-6713/PFI2',
 '/PXI-6713/PFI3',
 '/PXI-6713/PFI4',
 '/PXI-6713/PFI5',
 '/PXI-6713/PFI6',
 '/PXI-6713/PFI7',
 '/PXI-6713/PFI8',
 '/PXI-6713/PFI9',
 '/PXI-6713/PXI_Trig0',
 '/PXI-6713/PXI_Trig1',
 '/PXI-6713/PXI_Trig2',
 '/PXI-6713/PXI_Trig3',
 '/PXI-6713/PXI_Trig4',
 '/PXI-6713/PXI_Trig5',
 '/PXI-6713/PXI_Trig7',
 '/PXI-6713/PXI_Star',
 '/PXI-6713/ao/SampleClock',
 '/PXI-6713/ao/StartTrigger',
 '/PXI-6713/20MHzTimebase',
 '/PXI-6713/ao/PauseTrigger',
 '/PXI-6713/ao/SampleClockTimebase',
 '/PXI-6713/Ctr0Out',
 '/PXI-6713/Ctr0Gate',
 '/PXI-6713/Ctr0Source',
 '/PXI-6713/Ctr0InternalOutput',
 '/PXI-6713/Ctr1Out',
 '/PXI-6713/Ctr1Gate',
 '/PXI-6713/Ctr1Source',
 '/PXI-6713/Ctr1InternalOutput',
 '/PXI-6713/PairedCtrInternalOutput',
 '/PXI-6713/PairedCtrOutputPulse',
 '/PXI-6713/MasterTimebase',
 '/PXI-6713/100kHzTimebase']

In [9]:
daq_outputs._ci_channels

['PXI-6713/ctr0', 'PXI-6713/ctr1']

# with start/stop

## create task

In [6]:
daq_outputs.add_task('ao_task')

In [158]:
daq_outputs.ao_task._task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION

In [159]:
daq_clock_rate = 10000

In [160]:
daq_outputs.ao_task.add_sample_clock(daq_clock_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=1)

## generate scan list

In [161]:
Vxlist = Vylist = np.linspace(0,4,64)

In [162]:
Vxy_scan_list = scan_pattern(Vxlist,Vylist)

## initialize scanners

In [163]:
traj = trajectory([0, 0],[0, 0], 2, 1/daq_clock_rate)
daq_outputs.ao_task.clock.samp_quant_samp_per_chan.set(traj.shape[1])

daq_outputs.ao_task._task.write(traj, auto_start=False)
daq_outputs.ao_task.start()
while not daq_outputs.ao_task._task.is_task_done():
    print(daq_outputs.ao_task._task.out_stream.total_samp_per_chan_generated)
    time.sleep(0.1)
daq_outputs.ao_task.stop()

curr_pos = [0, 0]

## Perform scan

In [154]:
for Vxy in Vxy_scan_list:
    traj = trajectory(Vxy,curr_pos, 2, 1/daq_clock_rate)
    daq_outputs.ao_task.clock.samp_quant_samp_per_chan.set(traj.shape[1])
    daq_outputs.ao_task._task.write(traj, auto_start=False)
    daq_outputs.ao_task.start()
    while not daq_outputs.ao_task._task.is_task_done():
        pass
    daq_outputs.ao_task.stop()
    curr_pos = Vxy
    print(Vxy)
traj = trajectory(Vxy_scan_list[0],curr_pos, 2, 1/daq_clock_rate)
daq_outputs.ao_task.clock.samp_quant_samp_per_chan.set(traj.shape[1])
daq_outputs.ao_task._task.write(traj, auto_start=True)
while not daq_outputs.ao_task._task.is_task_done():
    time.sleep(0.1)
daq_outputs.ao_task.stop()

[0.0, 0.0]
[0.06349206349206349, 0.0]
[0.12698412698412698, 0.0]
[0.19047619047619047, 0.0]
[0.25396825396825395, 0.0]
[0.31746031746031744, 0.0]
[0.38095238095238093, 0.0]
[0.4444444444444444, 0.0]
[0.5079365079365079, 0.0]
[0.5714285714285714, 0.0]
[0.6349206349206349, 0.0]
[0.6984126984126984, 0.0]
[0.7619047619047619, 0.0]
[0.8253968253968254, 0.0]
[0.8888888888888888, 0.0]
[0.9523809523809523, 0.0]
[1.0158730158730158, 0.0]
[1.0793650793650793, 0.0]
[1.1428571428571428, 0.0]
[1.2063492063492063, 0.0]
[1.2698412698412698, 0.0]
[1.3333333333333333, 0.0]
[1.3968253968253967, 0.0]
[1.4603174603174602, 0.0]
[1.5238095238095237, 0.0]
[1.5873015873015872, 0.0]
[1.6507936507936507, 0.0]
[1.7142857142857142, 0.0]
[1.7777777777777777, 0.0]
[1.8412698412698412, 0.0]
[1.9047619047619047, 0.0]
[1.9682539682539681, 0.0]
[2.0317460317460316, 0.0]
[2.095238095238095, 0.0]
[2.1587301587301586, 0.0]
[2.2222222222222223, 0.0]
[2.2857142857142856, 0.0]
[2.349206349206349, 0.0]
[2.4126984126984126, 0.

[1.9047619047619047, 0.19047619047619047]
[1.8412698412698412, 0.19047619047619047]
[1.7777777777777777, 0.19047619047619047]
[1.7142857142857142, 0.19047619047619047]
[1.6507936507936507, 0.19047619047619047]
[1.5873015873015872, 0.19047619047619047]
[1.5238095238095237, 0.19047619047619047]
[1.4603174603174602, 0.19047619047619047]
[1.3968253968253967, 0.19047619047619047]
[1.3333333333333333, 0.19047619047619047]
[1.2698412698412698, 0.19047619047619047]
[1.2063492063492063, 0.19047619047619047]
[1.1428571428571428, 0.19047619047619047]
[1.0793650793650793, 0.19047619047619047]
[1.0158730158730158, 0.19047619047619047]
[0.9523809523809523, 0.19047619047619047]
[0.8888888888888888, 0.19047619047619047]
[0.8253968253968254, 0.19047619047619047]
[0.7619047619047619, 0.19047619047619047]
[0.6984126984126984, 0.19047619047619047]
[0.6349206349206349, 0.19047619047619047]
[0.5714285714285714, 0.19047619047619047]
[0.5079365079365079, 0.19047619047619047]
[0.4444444444444444, 0.19047619047

[2.6666666666666665, 0.38095238095238093]
[2.7301587301587302, 0.38095238095238093]
[2.7936507936507935, 0.38095238095238093]
[2.8571428571428568, 0.38095238095238093]
[2.9206349206349205, 0.38095238095238093]
[2.984126984126984, 0.38095238095238093]
[3.0476190476190474, 0.38095238095238093]
[3.1111111111111107, 0.38095238095238093]
[3.1746031746031744, 0.38095238095238093]
[3.238095238095238, 0.38095238095238093]
[3.3015873015873014, 0.38095238095238093]
[3.3650793650793647, 0.38095238095238093]
[3.4285714285714284, 0.38095238095238093]
[3.492063492063492, 0.38095238095238093]
[3.5555555555555554, 0.38095238095238093]
[3.6190476190476186, 0.38095238095238093]
[3.6825396825396823, 0.38095238095238093]
[3.746031746031746, 0.38095238095238093]
[3.8095238095238093, 0.38095238095238093]
[3.8730158730158726, 0.38095238095238093]
[3.9365079365079363, 0.38095238095238093]
[4.0, 0.38095238095238093]
[4.0, 0.4444444444444444]
[3.9365079365079363, 0.4444444444444444]
[3.8730158730158726, 0.44444

[0.6984126984126984, 0.5714285714285714]
[0.6349206349206349, 0.5714285714285714]
[0.5714285714285714, 0.5714285714285714]
[0.5079365079365079, 0.5714285714285714]
[0.4444444444444444, 0.5714285714285714]
[0.38095238095238093, 0.5714285714285714]
[0.31746031746031744, 0.5714285714285714]
[0.25396825396825395, 0.5714285714285714]
[0.19047619047619047, 0.5714285714285714]
[0.12698412698412698, 0.5714285714285714]
[0.06349206349206349, 0.5714285714285714]
[0.0, 0.5714285714285714]
[0.0, 0.6349206349206349]
[0.06349206349206349, 0.6349206349206349]
[0.12698412698412698, 0.6349206349206349]
[0.19047619047619047, 0.6349206349206349]
[0.25396825396825395, 0.6349206349206349]
[0.31746031746031744, 0.6349206349206349]
[0.38095238095238093, 0.6349206349206349]
[0.4444444444444444, 0.6349206349206349]
[0.5079365079365079, 0.6349206349206349]
[0.5714285714285714, 0.6349206349206349]
[0.6349206349206349, 0.6349206349206349]
[0.6984126984126984, 0.6349206349206349]
[0.7619047619047619, 0.63492063492

[4.0, 0.8253968253968254]
[3.9365079365079363, 0.8253968253968254]
[3.8730158730158726, 0.8253968253968254]
[3.8095238095238093, 0.8253968253968254]
[3.746031746031746, 0.8253968253968254]
[3.6825396825396823, 0.8253968253968254]
[3.6190476190476186, 0.8253968253968254]
[3.5555555555555554, 0.8253968253968254]
[3.492063492063492, 0.8253968253968254]
[3.4285714285714284, 0.8253968253968254]
[3.3650793650793647, 0.8253968253968254]
[3.3015873015873014, 0.8253968253968254]
[3.238095238095238, 0.8253968253968254]
[3.1746031746031744, 0.8253968253968254]
[3.1111111111111107, 0.8253968253968254]
[3.0476190476190474, 0.8253968253968254]
[2.984126984126984, 0.8253968253968254]
[2.9206349206349205, 0.8253968253968254]
[2.8571428571428568, 0.8253968253968254]
[2.7936507936507935, 0.8253968253968254]
[2.7301587301587302, 0.8253968253968254]
[2.6666666666666665, 0.8253968253968254]
[2.603174603174603, 0.8253968253968254]
[2.5396825396825395, 0.8253968253968254]
[2.4761904761904763, 0.8253968253968

[1.0158730158730158, 1.0158730158730158]
[1.0793650793650793, 1.0158730158730158]
[1.1428571428571428, 1.0158730158730158]
[1.2063492063492063, 1.0158730158730158]
[1.2698412698412698, 1.0158730158730158]
[1.3333333333333333, 1.0158730158730158]
[1.3968253968253967, 1.0158730158730158]
[1.4603174603174602, 1.0158730158730158]
[1.5238095238095237, 1.0158730158730158]
[1.5873015873015872, 1.0158730158730158]
[1.6507936507936507, 1.0158730158730158]
[1.7142857142857142, 1.0158730158730158]
[1.7777777777777777, 1.0158730158730158]
[1.8412698412698412, 1.0158730158730158]
[1.9047619047619047, 1.0158730158730158]
[1.9682539682539681, 1.0158730158730158]
[2.0317460317460316, 1.0158730158730158]
[2.095238095238095, 1.0158730158730158]
[2.1587301587301586, 1.0158730158730158]
[2.2222222222222223, 1.0158730158730158]
[2.2857142857142856, 1.0158730158730158]
[2.349206349206349, 1.0158730158730158]
[2.4126984126984126, 1.0158730158730158]
[2.4761904761904763, 1.0158730158730158]
[2.539682539682539

[1.9682539682539681, 1.2063492063492063]
[1.9047619047619047, 1.2063492063492063]
[1.8412698412698412, 1.2063492063492063]
[1.7777777777777777, 1.2063492063492063]
[1.7142857142857142, 1.2063492063492063]
[1.6507936507936507, 1.2063492063492063]
[1.5873015873015872, 1.2063492063492063]
[1.5238095238095237, 1.2063492063492063]
[1.4603174603174602, 1.2063492063492063]
[1.3968253968253967, 1.2063492063492063]
[1.3333333333333333, 1.2063492063492063]
[1.2698412698412698, 1.2063492063492063]
[1.2063492063492063, 1.2063492063492063]
[1.1428571428571428, 1.2063492063492063]
[1.0793650793650793, 1.2063492063492063]
[1.0158730158730158, 1.2063492063492063]
[0.9523809523809523, 1.2063492063492063]
[0.8888888888888888, 1.2063492063492063]
[0.8253968253968254, 1.2063492063492063]
[0.7619047619047619, 1.2063492063492063]
[0.6984126984126984, 1.2063492063492063]
[0.6349206349206349, 1.2063492063492063]
[0.5714285714285714, 1.2063492063492063]
[0.5079365079365079, 1.2063492063492063]
[0.4444444444444

[2.984126984126984, 1.3968253968253967]
[3.0476190476190474, 1.3968253968253967]
[3.1111111111111107, 1.3968253968253967]
[3.1746031746031744, 1.3968253968253967]
[3.238095238095238, 1.3968253968253967]
[3.3015873015873014, 1.3968253968253967]
[3.3650793650793647, 1.3968253968253967]
[3.4285714285714284, 1.3968253968253967]
[3.492063492063492, 1.3968253968253967]
[3.5555555555555554, 1.3968253968253967]
[3.6190476190476186, 1.3968253968253967]
[3.6825396825396823, 1.3968253968253967]
[3.746031746031746, 1.3968253968253967]
[3.8095238095238093, 1.3968253968253967]
[3.8730158730158726, 1.3968253968253967]
[3.9365079365079363, 1.3968253968253967]
[4.0, 1.3968253968253967]
[4.0, 1.4603174603174602]
[3.9365079365079363, 1.4603174603174602]
[3.8730158730158726, 1.4603174603174602]
[3.8095238095238093, 1.4603174603174602]
[3.746031746031746, 1.4603174603174602]
[3.6825396825396823, 1.4603174603174602]
[3.6190476190476186, 1.4603174603174602]
[3.5555555555555554, 1.4603174603174602]
[3.4920634

[0.06349206349206349, 1.5873015873015872]
[0.0, 1.5873015873015872]
[0.0, 1.6507936507936507]
[0.06349206349206349, 1.6507936507936507]
[0.12698412698412698, 1.6507936507936507]
[0.19047619047619047, 1.6507936507936507]
[0.25396825396825395, 1.6507936507936507]
[0.31746031746031744, 1.6507936507936507]
[0.38095238095238093, 1.6507936507936507]
[0.4444444444444444, 1.6507936507936507]
[0.5079365079365079, 1.6507936507936507]
[0.5714285714285714, 1.6507936507936507]
[0.6349206349206349, 1.6507936507936507]
[0.6984126984126984, 1.6507936507936507]
[0.7619047619047619, 1.6507936507936507]
[0.8253968253968254, 1.6507936507936507]
[0.8888888888888888, 1.6507936507936507]
[0.9523809523809523, 1.6507936507936507]
[1.0158730158730158, 1.6507936507936507]
[1.0793650793650793, 1.6507936507936507]
[1.1428571428571428, 1.6507936507936507]
[1.2063492063492063, 1.6507936507936507]
[1.2698412698412698, 1.6507936507936507]
[1.3333333333333333, 1.6507936507936507]
[1.3968253968253967, 1.6507936507936507

KeyboardInterrupt: 

In [79]:
traj = trajectory(Vxy_scan_list[-2],Vxy_scan_list[0], 0.5, 1/daq_clock_rate)

In [80]:
daq_outputs.ao_task.clock.samp_quant_samp_per_chan.set(traj.shape[1])

In [81]:
daq_outputs.ao_task._task.write(traj)

801

In [82]:
daq_outputs.ao_task.start()

In [83]:
daq_outputs.ao_task._task.is_task_done()

True

In [134]:
daq_outputs.ao_task._task.out_stream.total_samp_per_chan_generated

800000

In [56]:
daq_outputs.ao_task._task.out_stream.curr_write_pos

800

In [84]:
daq_outputs.ao_task.stop()

In [164]:
daq_outputs.ao_task.close()

# with pause triggers

In [78]:
daq_outputs.add_task('do_task')

In [79]:
ao_gating = daq_outputs.do_task._task.do_channels.add_do_chan('PXI-6713/port0/line0', 'ao_gating')

In [86]:
daq_outputs.do_task._task.write(False)

1

In [116]:
daq_outputs.do_task._task.export_signals

In [131]:
daq_outputs.add_task('ao_task')

In [132]:
daq_outputs.ao_task.add_ao_volt_channel('X','PXI-6713/ao0',
            min_val=0,
            max_val=4)

daq_outputs.ao_task.add_ao_volt_channel('Y','PXI-6713/ao1',
            min_val=0,
            max_val=4)

In [133]:
daq_outputs.ao_task._task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION

In [134]:
daq_clock_rate = 100

In [135]:
daq_outputs.ao_task.add_sample_clock(daq_clock_rate, sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=800)

In [136]:
daq_outputs.ao_task._task.triggers.pause_trigger.trig_type = TriggerType.DIGITAL_LEVEL

In [137]:
daq_outputs.ao_task._task.triggers.pause_trigger.dig_lvl_src = '/PXI-6713/PFI1'

In [138]:
daq_outputs.ao_task._task.triggers.pause_trigger.dig_lvl_when = Level.LOW

In [ ]:
daq_outputs.ao_task._task.register_every_n_samples_transferred_from_buffer_event()

In [139]:
Vxlist = Vylist = np.linspace(0,4,200)

In [140]:
Vxy_scan_list = scan_pattern(Vxlist,Vylist)

Exception ignored in: <function Task.__del__ at 0x0000026147D754C8>
Traceback (most recent call last):
  File "C:\Users\He5user\Anaconda3\envs\qcodes\lib\site-packages\nidaqmx\task.py", line 105, in __del__
    'reserved.'.format(self.name), DaqResourceWarning)
  File "C:\Users\He5user\Anaconda3\envs\qcodes\lib\site-packages\nidaqmx\task.py", line 156, in name
    check_for_error(size_or_code)
  File "C:\Users\He5user\Anaconda3\envs\qcodes\lib\site-packages\nidaqmx\errors.py", line 127, in check_for_error
    raise DaqError(error_buffer.value.decode("utf-8"), error_code)
nidaqmx.errors.DaqError: Task specified is invalid or does not exist.
Status Code: -200088


In [141]:
traj = trajectory(Vxy_scan_list[-2],Vxy_scan_list[0], 0.5, 1/daq_clock_rate)

In [142]:
daq_outputs.ao_task.clock.samp_quant_samp_per_chan.set(traj.shape[1])
daq_outputs.ao_task.clock.samp_quant_samp_per_chan()

800

In [153]:
daq_outputs.ao_task._task.write(traj)

800

In [154]:
daq_outputs.ao_task.start()

In [155]:
daq_outputs.do_task._task.write(True)

1

In [158]:
daq_outputs.ao_task._task.out_stream.total_samp_per_chan_generated

676

In [159]:
daq_outputs.ao_task._task.is_task_done()

DaqError: The generation has stopped to prevent the regeneration of old samples. Your application was unable to write samples to the background buffer fast enough to prevent old samples from being regenerated.

To avoid this error, you can do any of the following:
1. Increase the size of the background buffer by configuring the buffer.
2. Increase the number of samples you write each time you invoke a write operation.
3. Write samples more often.
4. Reduce the sample rate.
5. If your data transfer method is interrupts, try using DMA or USB Bulk.
6. Reduce the number of applications your computer is executing concurrently.

In addition, if you do not need to write every sample that is generated, you can configure the regeneration mode to allow regeneration, and then use the Position and Offset attributes to write the desired samples.
Task Name: ao_task

Status Code: -200290

In [157]:
daq_outputs.ao_task._task.out_stream.curr_write_pos

800

In [149]:
daq_outputs.do_task._task.write(False)

1

In [152]:
daq_outputs.ao_task.stop()

In [160]:
daq_outputs.ao_task.close()

# with counter 

In [138]:
daq_outputs.add_task("co_task")

daq_outputs.co_task.add_co_freq_channel('co', 'PXI-6713/ctr0', freq=10)

In [139]:
daq_outputs.co_task.add_implicit_clock(sample_mode=AcquisitionType.FINITE, samps_per_chan=100)

In [140]:
daq_outputs.co_task.start()

In [141]:
daq_outputs.co_task._task.timing.samp_quant_samp_per_chan=10

DaqError: Specified property cannot be set while the task is running.

Set the property prior to starting the task, or stop the task prior to setting the property.
Property: DAQmx_SampQuant_SampPerChan

Task Name: co_task

Status Code: -200557

In [129]:
daq_outputs.co_task._task.write(CtrFreq(0.5,100))

DaqError: The task is not buffered or has no channels.  If the task is not buffered, use the scalar version of this function. If the task has no channels, add one to the task.
Task Name: co_task

Status Code: -201395

In [137]:
daq_outputs.co_task.close()

# with Meta Instrument

## create Instrument

In [5]:
scan_driver = MetaXYScannerDriver('scan_driver', 'PXI-6713', ('PXI-6713/ao0','PXI-6713/ao1'), 10000, 2)

In [6]:
scan_driver.x_min.set(0)
scan_driver.x_max.set(4)
scan_driver.y_min.set(0)
scan_driver.y_max.set(4)

## generate scan list

In [7]:
Vxlist = Vylist = np.linspace(0,4,64)

In [8]:
Vxy_scan_list = scan_pattern(Vxlist,Vylist)

## initialize scanners

In [9]:
scan_driver.go_to((0,0))

In [10]:
scan_driver.scan_init(Vxlist, Vylist)

## Perform scan

In [10]:
for Vxy in Vxy_scan_list:
    scan_driver.go_to_smooth(Vxy)
    while not scan_driver.is_done_moving():
        pass
    print(scan_driver._pos)
    
scan_driver.go_to_smooth(Vxy_scan_list[0])
while not scan_driver.is_done_moving():
    pass

[0.0, 0.0]
[0.06349206349206349, 0.0]
[0.12698412698412698, 0.0]
[0.19047619047619047, 0.0]
[0.25396825396825395, 0.0]
[0.31746031746031744, 0.0]
[0.38095238095238093, 0.0]
[0.4444444444444444, 0.0]
[0.5079365079365079, 0.0]
[0.5714285714285714, 0.0]
[0.6349206349206349, 0.0]
[0.6984126984126984, 0.0]
[0.7619047619047619, 0.0]
[0.8253968253968254, 0.0]
[0.8888888888888888, 0.0]
[0.9523809523809523, 0.0]
[1.0158730158730158, 0.0]
[1.0793650793650793, 0.0]
[1.1428571428571428, 0.0]
[1.2063492063492063, 0.0]
[1.2698412698412698, 0.0]
[1.3333333333333333, 0.0]
[1.3968253968253967, 0.0]
[1.4603174603174602, 0.0]
[1.5238095238095237, 0.0]
[1.5873015873015872, 0.0]
[1.6507936507936507, 0.0]
[1.7142857142857142, 0.0]
[1.7777777777777777, 0.0]
[1.8412698412698412, 0.0]
[1.9047619047619047, 0.0]
[1.9682539682539681, 0.0]
[2.0317460317460316, 0.0]
[2.095238095238095, 0.0]
[2.1587301587301586, 0.0]
[2.2222222222222223, 0.0]
[2.2857142857142856, 0.0]
[2.349206349206349, 0.0]
[2.4126984126984126, 0.

[2.0317460317460316, 0.19047619047619047]
[1.9682539682539681, 0.19047619047619047]
[1.9047619047619047, 0.19047619047619047]
[1.8412698412698412, 0.19047619047619047]
[1.7777777777777777, 0.19047619047619047]
[1.7142857142857142, 0.19047619047619047]
[1.6507936507936507, 0.19047619047619047]
[1.5873015873015872, 0.19047619047619047]
[1.5238095238095237, 0.19047619047619047]
[1.4603174603174602, 0.19047619047619047]
[1.3968253968253967, 0.19047619047619047]
[1.3333333333333333, 0.19047619047619047]
[1.2698412698412698, 0.19047619047619047]
[1.2063492063492063, 0.19047619047619047]
[1.1428571428571428, 0.19047619047619047]
[1.0793650793650793, 0.19047619047619047]
[1.0158730158730158, 0.19047619047619047]
[0.9523809523809523, 0.19047619047619047]
[0.8888888888888888, 0.19047619047619047]
[0.8253968253968254, 0.19047619047619047]
[0.7619047619047619, 0.19047619047619047]
[0.6984126984126984, 0.19047619047619047]
[0.6349206349206349, 0.19047619047619047]
[0.5714285714285714, 0.19047619047

[2.4761904761904763, 0.38095238095238093]
[2.5396825396825395, 0.38095238095238093]
[2.603174603174603, 0.38095238095238093]
[2.6666666666666665, 0.38095238095238093]
[2.7301587301587302, 0.38095238095238093]
[2.7936507936507935, 0.38095238095238093]
[2.8571428571428568, 0.38095238095238093]
[2.9206349206349205, 0.38095238095238093]
[2.984126984126984, 0.38095238095238093]
[3.0476190476190474, 0.38095238095238093]
[3.1111111111111107, 0.38095238095238093]
[3.1746031746031744, 0.38095238095238093]
[3.238095238095238, 0.38095238095238093]
[3.3015873015873014, 0.38095238095238093]
[3.3650793650793647, 0.38095238095238093]
[3.4285714285714284, 0.38095238095238093]
[3.492063492063492, 0.38095238095238093]
[3.5555555555555554, 0.38095238095238093]
[3.6190476190476186, 0.38095238095238093]
[3.6825396825396823, 0.38095238095238093]
[3.746031746031746, 0.38095238095238093]
[3.8095238095238093, 0.38095238095238093]
[3.8730158730158726, 0.38095238095238093]
[3.9365079365079363, 0.3809523809523809

[0.8888888888888888, 0.5714285714285714]
[0.8253968253968254, 0.5714285714285714]
[0.7619047619047619, 0.5714285714285714]
[0.6984126984126984, 0.5714285714285714]
[0.6349206349206349, 0.5714285714285714]
[0.5714285714285714, 0.5714285714285714]
[0.5079365079365079, 0.5714285714285714]
[0.4444444444444444, 0.5714285714285714]
[0.38095238095238093, 0.5714285714285714]
[0.31746031746031744, 0.5714285714285714]
[0.25396825396825395, 0.5714285714285714]
[0.19047619047619047, 0.5714285714285714]
[0.12698412698412698, 0.5714285714285714]
[0.06349206349206349, 0.5714285714285714]
[0.0, 0.5714285714285714]
[0.0, 0.6349206349206349]
[0.06349206349206349, 0.6349206349206349]
[0.12698412698412698, 0.6349206349206349]
[0.19047619047619047, 0.6349206349206349]
[0.25396825396825395, 0.6349206349206349]
[0.31746031746031744, 0.6349206349206349]
[0.38095238095238093, 0.6349206349206349]
[0.4444444444444444, 0.6349206349206349]
[0.5079365079365079, 0.6349206349206349]
[0.5714285714285714, 0.63492063492

[3.8095238095238093, 0.7619047619047619]
[3.8730158730158726, 0.7619047619047619]
[3.9365079365079363, 0.7619047619047619]
[4.0, 0.7619047619047619]
[4.0, 0.8253968253968254]
[3.9365079365079363, 0.8253968253968254]
[3.8730158730158726, 0.8253968253968254]
[3.8095238095238093, 0.8253968253968254]
[3.746031746031746, 0.8253968253968254]
[3.6825396825396823, 0.8253968253968254]
[3.6190476190476186, 0.8253968253968254]
[3.5555555555555554, 0.8253968253968254]
[3.492063492063492, 0.8253968253968254]
[3.4285714285714284, 0.8253968253968254]
[3.3650793650793647, 0.8253968253968254]
[3.3015873015873014, 0.8253968253968254]
[3.238095238095238, 0.8253968253968254]
[3.1746031746031744, 0.8253968253968254]
[3.1111111111111107, 0.8253968253968254]
[3.0476190476190474, 0.8253968253968254]
[2.984126984126984, 0.8253968253968254]
[2.9206349206349205, 0.8253968253968254]
[2.8571428571428568, 0.8253968253968254]
[2.7936507936507935, 0.8253968253968254]
[2.7301587301587302, 0.8253968253968254]
[2.666666

...

[0.7619047619047619, 3.0476190476190474]
[0.8253968253968254, 3.0476190476190474]
[0.8888888888888888, 3.0476190476190474]
[0.9523809523809523, 3.0476190476190474]
[1.0158730158730158, 3.0476190476190474]
[1.0793650793650793, 3.0476190476190474]
[1.1428571428571428, 3.0476190476190474]
[1.2063492063492063, 3.0476190476190474]
[1.2698412698412698, 3.0476190476190474]
[1.3333333333333333, 3.0476190476190474]
[1.3968253968253967, 3.0476190476190474]
[1.4603174603174602, 3.0476190476190474]
[1.5238095238095237, 3.0476190476190474]
[1.5873015873015872, 3.0476190476190474]
[1.6507936507936507, 3.0476190476190474]
[1.7142857142857142, 3.0476190476190474]
[1.7777777777777777, 3.0476190476190474]
[1.8412698412698412, 3.0476190476190474]
[1.9047619047619047, 3.0476190476190474]
[1.9682539682539681, 3.0476190476190474]
[2.0317460317460316, 3.0476190476190474]
[2.095238095238095, 3.0476190476190474]
[2.1587301587301586, 3.0476190476190474]
[2.2222222222222223, 3.0476190476190474]
[2.28571428571428

[2.349206349206349, 3.238095238095238]
[2.2857142857142856, 3.238095238095238]
[2.2222222222222223, 3.238095238095238]
[2.1587301587301586, 3.238095238095238]
[2.095238095238095, 3.238095238095238]
[2.0317460317460316, 3.238095238095238]
[1.9682539682539681, 3.238095238095238]
[1.9047619047619047, 3.238095238095238]
[1.8412698412698412, 3.238095238095238]
[1.7777777777777777, 3.238095238095238]
[1.7142857142857142, 3.238095238095238]
[1.6507936507936507, 3.238095238095238]
[1.5873015873015872, 3.238095238095238]
[1.5238095238095237, 3.238095238095238]
[1.4603174603174602, 3.238095238095238]
[1.3968253968253967, 3.238095238095238]
[1.3333333333333333, 3.238095238095238]
[1.2698412698412698, 3.238095238095238]
[1.2063492063492063, 3.238095238095238]
[1.1428571428571428, 3.238095238095238]
[1.0793650793650793, 3.238095238095238]
[1.0158730158730158, 3.238095238095238]
[0.9523809523809523, 3.238095238095238]
[0.8888888888888888, 3.238095238095238]
[0.8253968253968254, 3.238095238095238]
[0

[2.603174603174603, 3.4285714285714284]
[2.6666666666666665, 3.4285714285714284]
[2.7301587301587302, 3.4285714285714284]
[2.7936507936507935, 3.4285714285714284]
[2.8571428571428568, 3.4285714285714284]
[2.9206349206349205, 3.4285714285714284]
[2.984126984126984, 3.4285714285714284]
[3.0476190476190474, 3.4285714285714284]
[3.1111111111111107, 3.4285714285714284]
[3.1746031746031744, 3.4285714285714284]
[3.238095238095238, 3.4285714285714284]
[3.3015873015873014, 3.4285714285714284]
[3.3650793650793647, 3.4285714285714284]
[3.4285714285714284, 3.4285714285714284]
[3.492063492063492, 3.4285714285714284]
[3.5555555555555554, 3.4285714285714284]
[3.6190476190476186, 3.4285714285714284]
[3.6825396825396823, 3.4285714285714284]
[3.746031746031746, 3.4285714285714284]
[3.8095238095238093, 3.4285714285714284]
[3.8730158730158726, 3.4285714285714284]
[3.9365079365079363, 3.4285714285714284]
[4.0, 3.4285714285714284]
[4.0, 3.492063492063492]
[3.9365079365079363, 3.492063492063492]
[3.873015873

[0.5079365079365079, 3.6190476190476186]
[0.4444444444444444, 3.6190476190476186]
[0.38095238095238093, 3.6190476190476186]
[0.31746031746031744, 3.6190476190476186]
[0.25396825396825395, 3.6190476190476186]
[0.19047619047619047, 3.6190476190476186]
[0.12698412698412698, 3.6190476190476186]
[0.06349206349206349, 3.6190476190476186]
[0.0, 3.6190476190476186]
[0.0, 3.6825396825396823]
[0.06349206349206349, 3.6825396825396823]
[0.12698412698412698, 3.6825396825396823]
[0.19047619047619047, 3.6825396825396823]
[0.25396825396825395, 3.6825396825396823]
[0.31746031746031744, 3.6825396825396823]
[0.38095238095238093, 3.6825396825396823]
[0.4444444444444444, 3.6825396825396823]
[0.5079365079365079, 3.6825396825396823]
[0.5714285714285714, 3.6825396825396823]
[0.6349206349206349, 3.6825396825396823]
[0.6984126984126984, 3.6825396825396823]
[0.7619047619047619, 3.6825396825396823]
[0.8253968253968254, 3.6825396825396823]
[0.8888888888888888, 3.6825396825396823]
[0.9523809523809523, 3.68253968253

[3.492063492063492, 3.8730158730158726]
[3.4285714285714284, 3.8730158730158726]
[3.3650793650793647, 3.8730158730158726]
[3.3015873015873014, 3.8730158730158726]
[3.238095238095238, 3.8730158730158726]
[3.1746031746031744, 3.8730158730158726]
[3.1111111111111107, 3.8730158730158726]
[3.0476190476190474, 3.8730158730158726]
[2.984126984126984, 3.8730158730158726]
[2.9206349206349205, 3.8730158730158726]
[2.8571428571428568, 3.8730158730158726]
[2.7936507936507935, 3.8730158730158726]
[2.7301587301587302, 3.8730158730158726]
[2.6666666666666665, 3.8730158730158726]
[2.603174603174603, 3.8730158730158726]
[2.5396825396825395, 3.8730158730158726]
[2.4761904761904763, 3.8730158730158726]
[2.4126984126984126, 3.8730158730158726]
[2.349206349206349, 3.8730158730158726]
[2.2857142857142856, 3.8730158730158726]
[2.2222222222222223, 3.8730158730158726]
[2.1587301587301586, 3.8730158730158726]
[2.095238095238095, 3.8730158730158726]
[2.0317460317460316, 3.8730158730158726]
[1.9682539682539681, 3

In [79]:
traj = trajectory(Vxy_scan_list[-2],Vxy_scan_list[0], 0.5, 1/daq_clock_rate)

In [80]:
daq_outputs.ao_task.clock.samp_quant_samp_per_chan.set(traj.shape[1])

In [81]:
daq_outputs.ao_task._task.write(traj)

801

In [82]:
daq_outputs.ao_task.start()

In [83]:
daq_outputs.ao_task._task.is_task_done()

True

In [134]:
daq_outputs.ao_task._task.out_stream.total_samp_per_chan_generated

800000

In [56]:
daq_outputs.ao_task._task.out_stream.curr_write_pos

800

In [84]:
daq_outputs.ao_task.stop()

In [164]:
daq_outputs.ao_task.close()